# Imports:

In [1]:
import numpy as np
import os

# Data Preparation:

In [2]:
root_dir = r"Data/"
file_names = os.listdir(root_dir)
datasets = {}
for file in file_names:
    with open(root_dir + file) as f:
        lines = f.readlines()
        for line in lines:
            if line.startswith("ITEMS SECTION"):
                data_start = lines.index(line) + 1
                break
            elif line.startswith("CAPACITY OF KNAPSACK:"):
                cap = int(line.split("\t")[-1])
        mat = np.zeros((len(lines) - data_start, 2), dtype=np.int32)
        for i in range(data_start, len(lines)):
            (_, mat[i - data_start, 0], mat[i - data_start, 1], _) = lines[i].split('\t')
        datasets[file] = mat, cap

# Define Functions:

In [3]:
def f(x, avg_w, values, _B, _ALPHA, _DELTA, _USE_CHEBYSHEV):
    x = np.array(x, dtype=np.float64)
    avg_w = np.array(avg_w, dtype=np.float64)
    exp_sum_w = np.dot(x, avg_w)
    sum_v = np.dot(x, values)
    sig_x_i = np.sum(x, dtype=np.float64)
    delta = _DELTA
    if _USE_CHEBYSHEV: # uniform Chebyshev:
        PWB = (delta ** 2.0) * sig_x_i / ((delta ** 2.0) * sig_x_i + 3.0 * (_B - exp_sum_w) ** 2.0)
    else: # chernoff:
        if sig_x_i != 0.0:
            denum_base = ((delta * sig_x_i + _B - exp_sum_w) / (delta * sig_x_i))
            PWB = (np.exp((_B - exp_sum_w) / (delta * sig_x_i)) / np.float_power(denum_base, denum_base)) ** (sig_x_i / 2.0)
            if np.isnan(PWB):
                PWB = 1.0
        else:
            PWB = 0.0
    return max(0.0, exp_sum_w - _B), max(0.0, PWB - _ALPHA), sum_v

def lex_compare(f1, f2):
    if f1[0] > f2[0] or (f1[0] == f2[0] and f1[1] > f2[1]) or (f1[0] == f2[0] and f1[1] == f2[1] and f1[2] < f2[2]):
        return 2
    else:
        return 1

# Main Algorithm Loop:

In [4]:
_MAX_ITER = 100000
_alpha_delta_settings = [
    (0.0001, 25),
    (0.0001, 50),
    (0.001, 25),
    (0.001, 50),
    (0.01, 25),
    (0.01, 50),
]
for dataset_idx in range(len(file_names)):
    print('|' * 120)
    print(file_names[dataset_idx])
    dataset = np.array(datasets[file_names[dataset_idx]][0])
    _B_ORIG = datasets[file_names[dataset_idx]][1]
    avg_w = dataset[:, 1]
    v = dataset[:, 0]
    n = dataset.shape[0]
    k = 0
    _GAMMA = 100
    remains = _B_ORIG
    idxs = np.argsort(avg_w)
    for i in idxs:
        if remains - avg_w[i] < 0:
            break
        remains -= avg_w[i]
        k += 1
    _B = _B_ORIG + k * _GAMMA
    for i in range(avg_w.shape[0]):
        avg_w[i] += _GAMMA
    for _ALPHA, _DELTA in _alpha_delta_settings:
        for _CHEBYSHEV in [False, True]:
            # Main loop
            results = []
            for _ in range(30):
                x = np.random.randint(0, 2, (n,))
                f_x = f(x, avg_w, v, _B, _ALPHA, _DELTA, _CHEBYSHEV)
                iter_ = 0
                for _ in range(_MAX_ITER):
                    y = np.random.binomial(1, 1.0 / n, (n,))
                    while (y == np.zeros_like(y)).all():
                        y = np.random.binomial(1, 1.0 / n, (n,))
                    y_ = np.logical_xor(x, y)
                    f_y = f(y_, avg_w, v, _B, _ALPHA, _DELTA, _CHEBYSHEV)
                    if lex_compare(f_y, f_x) == 1:
                        x = y_
                        f_x = f_y
                results.append(f_x[2])
                if f_x[0] != 0:
                    print("KNAPSACK Broken Error:", f_x)
            if _CHEBYSHEV:
                mod_str = "Chebyshev"
            else:
                mod_str = "Chernoff "
            print("_ALPHA:", _ALPHA, "\t_DELTA:", _DELTA, "\tMode:", mod_str, "\tProfit:", np.mean(results))
            print(results)
            print('     ' + '=' * 110)

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
eil101_n100_bounded-strongly-corr_01[1].ttp


<ipython-input-3-7d940d4e65ac>:13: RuntimeWarning: invalid value encountered in float_power
  PWB = (np.exp((_B - exp_sum_w) / (delta * sig_x_i)) / np.float_power(denum_base, denum_base)) ** (sig_x_i / 2.0)
<ipython-input-3-7d940d4e65ac>:13: RuntimeWarning: invalid value encountered in double_scalars
  PWB = (np.exp((_B - exp_sum_w) / (delta * sig_x_i)) / np.float_power(denum_base, denum_base)) ** (sig_x_i / 2.0)


_ALPHA: 0.0001 	_DELTA: 25 	Mode: Chernoff  	Profit: 14735.466666666667
[14846.0, 14577.0, 14747.0, 14851.0, 14530.0, 14755.0, 14872.0, 14805.0, 14754.0, 14640.0, 14730.0, 14752.0, 14427.0, 14663.0, 14689.0, 14505.0, 14909.0, 14827.0, 14763.0, 14952.0, 14652.0, 14591.0, 14616.0, 14636.0, 14836.0, 14820.0, 14921.0, 14652.0, 14844.0, 14902.0]
_ALPHA: 0.0001 	_DELTA: 25 	Mode: Chebyshev 	Profit: 9882.3
[9682.0, 9800.0, 10009.0, 9631.0, 9879.0, 9753.0, 9755.0, 10000.0, 10006.0, 9854.0, 9686.0, 9921.0, 9939.0, 9873.0, 9873.0, 10116.0, 9957.0, 9925.0, 9892.0, 10032.0, 10080.0, 10045.0, 9890.0, 9752.0, 9890.0, 9732.0, 9947.0, 9956.0, 9649.0, 9945.0]
_ALPHA: 0.0001 	_DELTA: 50 	Mode: Chernoff  	Profit: 13761.2
[13729.0, 13948.0, 13832.0, 13900.0, 13698.0, 13863.0, 13952.0, 13384.0, 13564.0, 13962.0, 13818.0, 13523.0, 13354.0, 13894.0, 13634.0, 13520.0, 13482.0, 13990.0, 13894.0, 13523.0, 13941.0, 13462.0, 13660.0, 13945.0, 13898.0, 13849.0, 13929.0, 13731.0, 13947.0, 14010.0]
_ALPHA: 0.0001 	_

<ipython-input-3-7d940d4e65ac>:13: RuntimeWarning: overflow encountered in double_scalars
  PWB = (np.exp((_B - exp_sum_w) / (delta * sig_x_i)) / np.float_power(denum_base, denum_base)) ** (sig_x_i / 2.0)


_ALPHA: 0.001 	_DELTA: 25 	Mode: Chernoff  	Profit: 14886.066666666668
[14960.0, 14827.0, 14868.0, 14691.0, 14988.0, 15059.0, 14867.0, 14796.0, 14890.0, 14746.0, 14880.0, 14642.0, 15042.0, 14892.0, 15058.0, 14511.0, 14947.0, 14949.0, 14965.0, 15079.0, 14931.0, 15049.0, 14878.0, 15002.0, 14765.0, 14491.0, 14805.0, 15053.0, 14977.0, 14974.0]
_ALPHA: 0.001 	_DELTA: 25 	Mode: Chebyshev 	Profit: 13443.366666666667
[13437.0, 13401.0, 13342.0, 13394.0, 13524.0, 13474.0, 13403.0, 13475.0, 13692.0, 13336.0, 13426.0, 13605.0, 13631.0, 13047.0, 13184.0, 13472.0, 13439.0, 13588.0, 13544.0, 13267.0, 13467.0, 13430.0, 13187.0, 13705.0, 13531.0, 13339.0, 13293.0, 13456.0, 13532.0, 13680.0]
_ALPHA: 0.001 	_DELTA: 50 	Mode: Chernoff  	Profit: 14100.1
[14058.0, 13852.0, 14252.0, 14199.0, 14118.0, 14028.0, 14338.0, 14066.0, 14077.0, 13885.0, 14236.0, 14075.0, 14096.0, 14177.0, 14078.0, 13911.0, 14307.0, 14178.0, 14041.0, 14278.0, 14206.0, 14113.0, 13647.0, 14078.0, 14309.0, 14041.0, 13980.0, 14314.0, 141